In [ ]:
import math
import matplotlib.pyplot as plt
import pandas as pd
import re

def get_config(file_name):
    with open(file_name) as log_file:
        for line in log_file:
            return line # first line.

def get_line_stats(stats, match, fold_type):
    if match:
        #fold = f'{fold_type} {match.group(1)}'
        fold = match.group(1)
        epoch = match.group(2)
        stat_name = match.group(3)
        value = match.group(4)

        if fold not in stats:
            stats[fold] = dict()

        #if stat_name not in stats[fold]:
        #    stats[fold][stat_name] = []
        if stat_name not in stats[fold]:
            stats[fold][stat_name] = dict()
            
        if fold_type not in stats[fold][stat_name]:
            stats[fold][stat_name][fold_type] = []

        stats[fold][stat_name][fold_type].append(value)
        
def get_train_stats(file_name):
    with open(file_name) as log_file:
        stats = dict()
        for line in log_file:
            train_match = re.search(r'Fold ([0-9]+) Epoch ([0-9]+).*Train stats ([a-zA-Z_]+): ([0-9\.e-]+)', line)
            get_line_stats(stats, train_match, 'Train')

            valid_match = re.search(r'Fold ([0-9]+) Epoch ([0-9]+).*Valid stats ([a-zA-Z_]+): ([0-9\.e-]+)', line)
            get_line_stats(stats, valid_match, 'Valid')

            all_match = re.search(r'(ALL) Epoch ([0-9]+).*Train stats ([a-zA-Z_]+): ([0-9\.e-]+)', line)
            get_line_stats(stats, all_match, 'Train')
    return stats
        
def draw_stats(stats):
    for fold, fold_stats in stats.items():
        fig = plt.figure(figsize=(12, 2))
        cols = len(fold_stats.keys())
        i = 1
        for stat_name, fold_types in fold_stats.items():
            subplot = fig.add_subplot(1, cols, i)
            i += 1
            max_y = 1.05
            for fold_type, values in fold_types.items():
                x = range(1, len(values)+1)
                y = [float(v) for v in values]
                plt.plot(x, y, marker='.', label=fold_type)
                max_y = max(y + [max_y])
            plt.ylim(0, max_y)
            plt.legend()
            plt.title(f'Fold {fold} {stat_name}')
        plt.show()

def get_test_stats(file_name):
    with open(file_name) as log_file:
        stats = dict()
        for line in log_file:
            match = re.search(r'\[TEST (.*)\] Stats ([a-zA-Z_]+): ([0-9\.e-]+)', line)
            if match:
                model = match.group(1)
                stat_name = match.group(2)
                value = match.group(3)
                
                if model not in stats:
                    stats[model] = dict()
                    
                stats[model][stat_name] = value
    return stats

def get_network(file_name):
    net_lines = []
    with open(file_name) as log_file:
        reading_net = False
        first_using = False
        for line in log_file:
            match = re.search(' Using ', line)
            if match:
                if not first_using:
                    first_using = True
                else:
                    reading_net = True
            else:
                match = re.search('Starting Epoch', line)
                if match:
                    reading_net = False
                    break
                
            if reading_net:
                net_lines.append(line)
    return net_lines
                

<h2>Stats from simple network training</h2>

In [ ]:
get_config('results/calcium_8/run_calcium_classifier.log')

In [ ]:
stats = get_train_stats('results/calcium_8/run_calcium_classifier.log')
draw_stats(stats)

In [ ]:
stats = get_test_stats('results/calcium_8/run_calcium_classifier.log')
pd.DataFrame(stats)

<h2>Stats from medicalnet 34</h2>

In [ ]:
get_config('results/calcium_3/run_calcium_classifier.log')

In [ ]:
stats = get_train_stats('results/calcium_3/run_calcium_classifier.log')
draw_stats(stats)

In [ ]:
stats = get_test_stats('results/calcium_3/run_calcium_classifier.log')
pd.DataFrame(stats)

<h2>Stats from medicalnet 18 training</h2>

In [ ]:
get_config('results/calcium_4/run_calcium_classifier.log')

In [ ]:
stats = get_train_stats('results/calcium_4/run_calcium_classifier.log')
draw_stats(stats)

In [ ]:
stats = get_test_stats('results/calcium_4/run_calcium_classifier.log')
pd.DataFrame(stats)

<h2>Stats from medicalnet 10 training</h2>

In [ ]:
get_config('results/calcium_5/run_calcium_classifier.log')

In [ ]:
stats = get_train_stats('results/calcium_5/run_calcium_classifier.log')
draw_stats(stats)

In [ ]:
stats = get_test_stats('results/calcium_5/run_calcium_classifier.log')
pd.DataFrame(stats)

<h2>Stats from medicalnet 10 training - with medicalnet norm + improved classifier layers</h2>

In [ ]:
get_config('results/calcium_6/run_calcium_classifier.log')

In [ ]:
stats = get_train_stats('results/calcium_6/run_calcium_classifier.log')
draw_stats(stats)

In [ ]:
stats = get_test_stats('results/calcium_6/run_calcium_classifier.log')
pd.DataFrame(stats)

<h2>Stats from medicalnet 10 training - with medicalnet norm</h2>

In [ ]:
log_file = 'results/calcium_17/run_calcium_classifier.log'
print(get_config(log_file))
get_network(log_file)

In [ ]:
stats = get_train_stats(log_file)
draw_stats(stats)

In [ ]:
stats = get_test_stats(log_file)
frame = pd.DataFrame(stats)
#frame[['last_model.pth']]
frame